In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
import torch_classes_v2
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb
import time

In [2]:
%env "WANDB_NOTEBOOK_NAME" os.path.basename(__file__)

env: "WANDB_NOTEBOOK_NAME"=os.path.basename(__file__)


In [3]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [5]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [6]:
import importlib

In [7]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

In [8]:
weights_df = pd.DataFrame(data=list(zip(range(0,201),weights)),columns=['stock_id','index_weight'])

In [9]:
train = train.merge(weights_df,on='stock_id')

In [10]:
train['wap_calc'] = (train['bid_price']*train['ask_size']+train['ask_price']*train['bid_size'])/(train['ask_size']+train['bid_size'])

In [11]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc'],
      dtype='object')

In [12]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'wap_t-60'] = df_g['wap'].shift(6)
    df[f'target_t-60'] = df_g['target'].shift(6)
    df[f'initial_wap'] = df_g['wap_calc'].transform('first')
    df[f'initial_bid_size'] = df_g['bid_size'].transform('first')
    df[f'initial_ask_size'] = df_g['ask_size'].transform('first')
    cols = ['bid_price','ask_price','bid_size','ask_size']
    for i in cols:
        df[f'{i}_t-60'] = df_g[i].shift(-6)

    return(df)

In [13]:
def generate_index(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    df[f'index_wap'] = df_g['wap_weighted'].transform('mean')
    return(df)

def generate_index_2(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    df[f'index_wap_t-60'] = df_g['index_wap'].shift(6)
    df[f'index_wap_init'] = df_g['index_wap'].transform('first')
    return(df)

def generate_index_3(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    df[f'index_wap_t-60'] = df_g['index_wap_move_to_init'].shift(6)
    return(df)

In [14]:
train['wap_weighted'] = train['wap']*train['index_weight']
train_g = train.groupby(['stock_id','date_id'])
train = generate_prev_race(train,train_g)
train['delta_wap'] = train['wap']/train['wap_t-60']

train_g = train.groupby(['seconds_in_bucket','date_id'])
train = generate_index(train,train_g)


train['wap_move_to_init'] = train['wap_calc']/train['initial_wap']
train_g = train.groupby(['date_id'])
train = generate_index_2(train,train_g)

train['index_wap_move_to_init'] = train['index_wap']/train['index_wap_init']
train_g = train.groupby(['date_id'])
train = generate_index_3(train,train_g)

In [15]:
train_g = train.groupby(['stock_id','date_id'])
train = generate_prev_race(train,train_g)
train['delta_wap'] = train['wap']/train['wap_t-60']

In [16]:
train_g = train.groupby(['seconds_in_bucket','date_id'])
train = generate_index(train,train_g)
train['delta_wap'] = (train['wap']/train['wap_t-60'])

In [17]:
train_g = train.groupby(['date_id'])
train = generate_index_2(train,train_g)
train = generate_index_3(train,train_g)
train['delta_wap'] = (train['wap']/train['wap_t-60'])

In [18]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc', 'wap_weighted', 'wap_t-60', 'target_t-60', 'initial_wap',
       'initial_bid_size', 'initial_ask_size', 'bid_price_t-60',
       'ask_price_t-60', 'bid_size_t-60', 'ask_size_t-60', 'delta_wap',
       'index_wap', 'wap_move_to_init', 'index_wap_t-60', 'index_wap_init',
       'index_wap_move_to_init'],
      dtype='object')

In [19]:
train['target_calc'] = -((train['wap_t-60']/train['wap'])-(train['index_wap_t-60']/train['index_wap_move_to_init']))*10000
train['target_delta'] = train['target_t-60']-train['target_calc']

In [20]:
train_stock_0 = train[train['stock_id']==0].dropna(subset='bid_size_t-60').copy()
train_stock_0.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,bid_size_t-60,ask_size_t-60,delta_wap,index_wap,wap_move_to_init,index_wap_t-60,index_wap_init,index_wap_move_to_init,target_calc,target_delta
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,...,14005.50,10085.04,NaN,0.005031,1.000000,NaN,0.005031,1.000000,NaN,NaN
1,0,0,10,1299772.70,1,1.000026,15261106.63,NaN,NaN,0.999812,...,37904.16,17366.82,NaN,0.005033,0.999892,NaN,0.005031,1.000356,NaN,NaN
2,0,0,20,1299772.70,1,0.999919,15261106.63,NaN,NaN,0.999812,...,9427.34,61984.40,NaN,0.005034,0.999842,NaN,0.005031,1.000525,NaN,NaN
3,0,0,30,1299772.70,1,1.000133,15261106.63,NaN,NaN,1.000026,...,23340.00,40433.54,NaN,0.005034,1.000085,NaN,0.005031,1.000547,NaN,NaN
4,0,0,40,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000241,...,9801.75,42572.16,NaN,0.005035,1.000317,NaN,0.005031,1.000635,NaN,NaN
5,0,0,50,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000348,...,5039.82,28375.36,NaN,0.005035,1.000435,NaN,0.005031,1.000668,NaN,NaN
6,0,0,60,1218204.43,1,1.000562,15342674.90,NaN,NaN,1.000455,...,43482.46,68224.23,1.000517,0.005036,1.000517,1.000000,0.005031,1.000815,-2.979650,-0.050054
7,0,0,70,1264494.89,1,1.000455,15352380.96,NaN,NaN,1.000348,...,32658.50,13999.50,1.000529,0.005036,1.000422,1.000356,0.005031,1.000843,0.420661,-0.030847
8,0,0,80,1189832.86,1,1.000241,15427043.00,NaN,NaN,1.000133,...,33028.20,25196.40,1.000306,0.005033,1.000148,1.000525,0.005031,1.000409,4.216233,0.003776
9,0,0,90,1189272.89,1,1.000562,15427602.97,NaN,NaN,1.000348,...,51688.20,15769.39,1.000341,0.005033,1.000427,1.000547,0.005031,1.000345,5.430774,0.019474


In [21]:
train[['seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',  
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 
        'wap', 'index_weight','wap_calc','initial_wap','wap_weighted', 'index_wap', 'index_wap_init', 'index_wap_move_to_init']].isna().sum()

seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
index_weight                     0
wap_calc                       220
initial_wap                    220
wap_weighted                   220
index_wap                        0
index_wap_init                   0
index_wap_move_to_init           0
dtype: int64

In [22]:
# train_stock_0.to_csv('train_with_new_vars_0stock.csv')

In [23]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id"
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [24]:
train['bid_price_target'] = train['bid_price']-train['bid_price_t-60']

In [25]:
train['bid_price_t-60'] = train['bid_price_target']*10_000

In [26]:
train[['bid_price_t-60','bid_price']].head(10)

,bid_price_t-60,bid_price
0,-6.43,0.999812
1,-5.36,0.999812
2,-3.21,0.999812
3,-3.22,1.000026
4,0.00,1.000241
5,3.22,1.000348
6,6.43,1.000455
7,5.36,1.000348
8,4.28,1.000133
9,6.43,1.000348


In [27]:
X = train
X = train.dropna(subset='bid_size_t-60').copy().reset_index()

In [28]:
null_data = X.query('stock_id == 184 and date_id == 192')

In [29]:
len(null_data)

49

In [30]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:21<03:35,  1.19s/it]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [01:51<01:27,  1.13it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [02:27<01:09,  1.01s/it]

...

 79%|███████▉  | 158/200 [03:07<00:39,  1.07it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [04:04<00:00,  1.22s/it]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 1358.50it/s]


In [31]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [32]:
optim_dict = {
    'RMSProp':optim.RMSprop,
    "Adam":optim.Adam,
    "AdamW":optim.AdamW,
    'SGD':optim.SGD,
    
}

In [39]:
def model_pipeline(trading_df=trading_data, config=None):
    trading_df = trading_data
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        config = wandb.config

        input_size = len(trading_df.stocksDict[0].data_daily[0][0])
        
        model = torch_classes.GRUNetV2(input_size,config['hidden_size'],num_layers=config['num_layers']).to('cuda:0')
        config = wandb.config
        optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
        # optimizer = optim.Adadelta(model.parameters())
        print(model)
        trading_df.reset_hidden(hidden_size=config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.MSELoss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [34]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [35]:
trading_data.reset_hidden(16,num_layers=5)

In [41]:
config_static = {'learning_rate':0.00001, 'hidden_size':128, 'num_layers':2, 'batch_norm':1}
config = config_static

In [42]:
model = model_pipeline(trading_data, config_static)

GRUNetV2(
  (gru): GRU(17, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=128, out_features=128, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=128, out_features=1, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

created path


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_l1_loss,▂▆██▅▃▄▂▃▄▁▂▄▄▄▆▄▂▂▂▅▅▅▃▃▅▄▃▂▃▂▃▃▄▄▂▃▃▁▂
epoch_loss,█▇▄▅▃▂▄▂▂▅▄▄▅▃▄▅▄▃▄▃▅▄▅▄▃▅▄▄▃▄▃▄▄▃▄▂▂▂▁▂
loss_1,▃▆▂▂▅▂▂▃▂▂▇▁▁▃▂▁▄▂▂▄▂▂▆▂▁▆▃▁▃▂▂▄▂▂█▁▁▃▃▁
output_sd,▁▄███▇▆▆▇▆▄▅▆▇▆▆▅▆▅▅▆▆▆▆▅▆▅▅▅▆▆▆▅▆▆▅▆▆▅▆
relu_sum,█▇▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂
val_epoch_loss,▁▆██▇▅▃▄▅▄▄▅▆▇▇▆▅▄▄▄▅▆▆▅▄▅▄▅▅▆▆▅▄▅▆▄▅▆▅▇
val_epoch_loss_l1,▁▇██▇▆▄▅▆▆▄▆▇▆▇▆▅▅▅▅▆▆▇▅▅▆▅▆▆▇▇▆▅▆▆▅▆▇▆▇
val_loss,▃█▆▇▆▅▅▅▄▅▄▄▄▄▁▁▂▂▂▃▄▆▇▅▄▅▄▄▃▃▃▂▁▂▃▂▂▃▃▃
epoch,9999
epoch_loss,0.97318


In [ ]:
model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=4)

In [ ]:
len(targets)

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=1)

stocks_hidden = [torch.stack(x) for x in stocks_hidden]

X = torch.cat(stocks_hidden,dim=-1)

In [ ]:
stocks_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = trading_data.stocksDict[i].lgbm_pred_daily[1]
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        

    stock_hidden.append([torch.cat((x,y.reshape(1)),0) for x,y in zip(trading_data.stocksDict[i].hidden_all,stock_lgbm)])

In [ ]:
stocks_hidden_og = stocks_hidden
X1 = torch.cat(stocks_hidden_og,dim=-1)

In [ ]:
stocks_hidden_og?

In [ ]:
X1?

In [ ]:
X1.shape

In [ ]:
stocks_hidden_og[0] == stocks_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = trading_data.stocksDict[i].lgbm_pred_daily[1]
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        

    stock_hidden.append([torch.cat((x,y.reshape(1)),0) for x,y in zip(trading_data.stocksDict[i].hidden_all,stock_lgbm)])



In [ ]:
stock_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = torch.stack(trading_data.stocksDict[i].lgbm_pred_daily[1]).reshape(-1,1)
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        
    # print(stock_lgbm.shape)
    # print(trading_data.stocksDict[i].hidden_all.shape)
    stock_hidden.append([torch.cat((trading_data.stocksDict[i].hidden_all,stock_lgbm),dim=1)])

In [ ]:
stocks_hidden_og = stocks_hidden
X1 = torch.cat(stocks_hidden_og,dim=-1)

In [ ]:
X1?

In [ ]:
stock_lgbm = torch.stack(trading_data.stocksDict[1].lgbm_pred_daily[1])
stock_lgbm.shape

In [ ]:
hidden = trading_data.stocksDict[1].hidden_all
hidden.shape

In [ ]:
torch.cat([hidden,stock_lgbm],dim=1)

In [ ]:
len(stock_targets)

In [ ]:
stocks_hidden = [torch.stack(x) for x in stocks_hidden]
X = torch.cat(stocks_hidden,dim=-1)
Y = torch.stack(targets).transpose(0,1).to('cuda:0')

In [ ]:
output, relu = model(X)

In [ ]:
output.shape

In [ ]:
Y.shape

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "val_epoch_loss", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"values": ["adamW", 'adam', 'SGD', 'RMSprop']},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2,3,4,5]},
    'hidden_size':{'values':[8,16,32,64,128,256,512,1024]},
    'learning_rate': {'max': 0.001, 'min': 0.00001},
    'batch_norm':{'values':[0,1,2]}
}

sweep_config["parameters"] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Optiver Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)